### Imports

In [4]:
# Data Handling
import pandas as pd
import numpy as np

# Financial Data
import yfinance as yf

# Technical Analysis
import ta

# Machine Learning
import sklearn

# Visualization
import matplotlib.pyplot as plt

# Backtesting
import backtrader as bt

### Top 100 Stocks over the past 5 years

In [44]:
import yfinance as yf
import pandas as pd

# List of the top 10 ticker symbols
ticker_symbols = [ "MSFT", "AAPL", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "SPY", "V", "AMD",
    "NVDA", "INTC", "CSCO", "WMT", "DIS", "MCD", "BA", "PYPL", "SNAP", "NKE",
    "XOM", "CVX", "JNJ", "PFE", "BABA", "T", "VZ", "GE", "IBM", "GS", "JPM",
    "MS", "GS", "UNH", "CVS", "HD", "LOW", "UPS", "SBUX", "BLK", "DELL", "LMT",
    "ABT", "TXN", "TMO", "MRK", "AMAT", "INTU", "ADI", "MU", "LULU", "PLUG",
    "F", "GM", "COF", "STZ", "KMX", "TSCO", "LMT", "SHOP", "SQ", "BIDU", "SPG",
    "MELI", "BA", "ATVI", "GS", "CAT", "HLT", "TGT", "DOW", "COP", "BA", "NKE",
    "LULU", "GS", "JNJ", "JPM", "BRK-A", "MSFT", "V", "TSLA", "XOM", "PFE",
    "WMT", "GE", "T", "VZ", "AMZN", "BABA", "CSCO", "NVDA", "AAPL", "GOOGL", "INTC",
    "AMT", "CVX", "META", "WFC", "KO", "PEP", "K", "SBUX", "NKE", "LOW", "MCD",
    "DIS", "WMT", "XOM", "UPS", "HD", "TMO", "PYPL", "UBER", "ZM", "TEAM", "MDB",
    "DKNG"]

# Create an empty DataFrame to store all the data
all_data = pd.DataFrame()

# Loop through each ticker symbol and fetch data
for ticker_symbol in ticker_symbols:
    # Create a Ticker object for each stock
    ticker = yf.Ticker(ticker_symbol)

    # Fetch historical market data from 2024 to the present
    historical_data = ticker.history(start="2020-01-01", end=pd.to_datetime("today").strftime('%Y-%m-%d'))

    # Add a new column 'Ticker' with the ticker symbol for each row
    historical_data['Ticker'] = ticker_symbol

    # Append the data to the all_data DataFrame
    all_data = pd.concat([all_data, historical_data])

# Reset the index after combining all the data
all_data.reset_index(inplace=True)

# Display the first few rows of the combined data
print(all_data[['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']])

# Optionally, save the data to a CSV file
all_data.to_csv('top_10_stocks_2024.csv', index=False)
print("Data saved to top_10_stocks_2024.csv")

$ATVI: possibly delisted; no timezone found
C:\Users\Sezy\AppData\Local\Temp\ipykernel_33544\2875241900.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_data = pd.concat([all_data, historical_data])


       Ticker                      Date        Open        High         Low  \
0        MSFT 2020-01-02 00:00:00-05:00  151.870762  153.735906  151.440346   
1        MSFT 2020-01-03 00:00:00-05:00  151.430809  152.989871  151.182113   
2        MSFT 2020-01-06 00:00:00-05:00  150.244705  152.176809  149.699501   
3        MSFT 2020-01-07 00:00:00-05:00  152.387267  152.722028  150.474296   
4        MSFT 2020-01-08 00:00:00-05:00  152.014192  153.802829  151.076840   
...       ...                       ...         ...         ...         ...   
149287   DKNG 2025-01-23 00:00:00-05:00   40.770000   41.535000   40.250000   
149288   DKNG 2025-01-24 00:00:00-05:00   41.570000   42.000000   41.090000   
149289   DKNG 2025-01-27 00:00:00-05:00   40.500000   41.455002   40.070000   
149290   DKNG 2025-01-28 00:00:00-05:00   41.250000   42.279999   41.230000   
149291   DKNG 2025-01-29 00:00:00-05:00   42.200001   42.305000   41.200001   

             Close      Volume  
0       153.630692

### UPDATE STOCKS INFO
Ran to update MongoDatabase daily, collecting most recent stock data

In [9]:
from datetime import datetime, timedelta
from pymongo import MongoClient
import yfinance as yf
from dateutil import parser

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["Stock_Data"]
collection = db["Stock_5year"]

# Get today's date
today = datetime.today().strftime('%Y-%m-%d')

# List of 100 ticker symbols (Your original list)
ticker_symbols = [ 
    "MSFT", "AAPL", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "SPY", "V", "AMD",
    "NVDA", "INTC", "CSCO", "WMT", "DIS", "MCD", "BA", "PYPL", "SNAP", "NKE",
    "XOM", "CVX", "JNJ", "PFE", "BABA", "T", "VZ", "GE", "IBM", "GS", "JPM",
    "MS", "GS", "UNH", "CVS", "HD", "LOW", "UPS", "SBUX", "BLK", "DELL", "LMT",
    "ABT", "TXN", "TMO", "MRK", "AMAT", "INTU", "ADI", "MU", "LULU", "PLUG",
    "F", "GM", "COF", "STZ", "KMX", "TSCO", "LMT", "SHOP", "SQ", "BIDU", "SPG",
    "MELI", "BA", "ATVI", "GS", "CAT", "HLT", "TGT", "DOW", "COP", "BA", "NKE",
    "LULU", "GS", "JNJ", "JPM", "BRK-A", "MSFT", "V", "TSLA", "XOM", "PFE",
    "WMT", "GE", "T", "VZ", "AMZN", "BABA", "CSCO", "NVDA", "AAPL", "GOOGL", "INTC",
    "AMT", "CVX", "META", "WFC", "KO", "PEP", "K", "SBUX", "NKE", "LOW", "MCD",
    "DIS", "WMT", "XOM", "UPS", "HD", "TMO", "PYPL", "UBER", "ZM", "TEAM", "MDB",
    "DKNG"
]

# Loop through each stock and update only missing data
for ticker_symbol in ticker_symbols:
    print(f"🔍 Checking latest data for {ticker_symbol}...")

    # Get the most recent date for this stock in MongoDB
    latest_entry = collection.find_one({"Ticker": ticker_symbol}, sort=[("Date", -1)])

    if latest_entry:
        last_date_in_db = latest_entry["Date"]
        # Check if the last_date_in_db is a string and parse it
        if isinstance(last_date_in_db, str):
            last_date_in_db = parser.parse(last_date_in_db)
    else:
        # If no records exist, start from 5 years ago
        last_date_in_db = datetime.today() - timedelta(days=5*365)

    # Fetch only new data since the last recorded date
    start_date = (last_date_in_db + timedelta(days=1)).strftime('%Y-%m-%d')

    # If start_date is today, no need to fetch (already up to date)
    if start_date >= today:
        print(f"✅ {ticker_symbol} is already up-to-date.")
        continue

    print(f"📈 Fetching new data for {ticker_symbol} from {start_date} to {today}...")

    ticker = yf.Ticker(ticker_symbol)
    new_data = ticker.history(start=start_date, end=today)

    if new_data.empty:
        print(f"⚠️ No new data found for {ticker_symbol}, skipping...")
        continue

    # Reset index & add ticker column
    new_data.reset_index(inplace=True)
    new_data["Ticker"] = ticker_symbol

    # Convert to dictionary format for MongoDB
    records = new_data.to_dict(orient='records')

    # Insert new records into MongoDB
    collection.insert_many(records)
    print(f"✅ Updated {len(records)} new entries for {ticker_symbol}")

print("🎉 Stock data update complete!")

🔍 Checking latest data for MSFT...
📈 Fetching new data for MSFT from 2025-02-01 to 2025-02-03...


$MSFT: possibly delisted; no price data found  (1d 2025-02-01 -> 2025-02-03)


⚠️ No new data found for MSFT, skipping...
🔍 Checking latest data for AAPL...
📈 Fetching new data for AAPL from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for AAPL
🔍 Checking latest data for GOOGL...
📈 Fetching new data for GOOGL from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for GOOGL
🔍 Checking latest data for AMZN...
📈 Fetching new data for AMZN from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for AMZN
🔍 Checking latest data for TSLA...
📈 Fetching new data for TSLA from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for TSLA
🔍 Checking latest data for META...
📈 Fetching new data for META from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for META
🔍 Checking latest data for NVDA...
📈 Fetching new data for NVDA from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for NVDA
🔍 Checking latest data for SPY...
📈 Fetching new data for SPY from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for SPY
🔍 Checking latest data for V...
📈 Fetching new 

$NVDA: possibly delisted; no price data found  (1d 2025-02-01 -> 2025-02-03)


📈 Fetching new data for NVDA from 2025-02-01 to 2025-02-03...
⚠️ No new data found for NVDA, skipping...
🔍 Checking latest data for INTC...
📈 Fetching new data for INTC from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for INTC
🔍 Checking latest data for CSCO...
📈 Fetching new data for CSCO from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for CSCO
🔍 Checking latest data for WMT...
📈 Fetching new data for WMT from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for WMT
🔍 Checking latest data for DIS...
📈 Fetching new data for DIS from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for DIS
🔍 Checking latest data for MCD...
📈 Fetching new data for MCD from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for MCD
🔍 Checking latest data for BA...
📈 Fetching new data for BA from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for BA
🔍 Checking latest data for PYPL...
📈 Fetching new data for PYPL from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for PYPL


...

⚠️ No new data found for UPS, skipping...
🔍 Checking latest data for HD...
📈 Fetching new data for HD from 2025-02-01 to 2025-02-03...


$HD: possibly delisted; no price data found  (1d 2025-02-01 -> 2025-02-03)
$TMO: possibly delisted; no price data found  (1d 2025-02-01 -> 2025-02-03)


⚠️ No new data found for HD, skipping...
🔍 Checking latest data for TMO...
📈 Fetching new data for TMO from 2025-02-01 to 2025-02-03...
⚠️ No new data found for TMO, skipping...
🔍 Checking latest data for PYPL...


$PYPL: possibly delisted; no price data found  (1d 2025-02-01 -> 2025-02-03)


📈 Fetching new data for PYPL from 2025-02-01 to 2025-02-03...
⚠️ No new data found for PYPL, skipping...
🔍 Checking latest data for UBER...
📈 Fetching new data for UBER from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for UBER
🔍 Checking latest data for ZM...
📈 Fetching new data for ZM from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for ZM
🔍 Checking latest data for TEAM...
📈 Fetching new data for TEAM from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for TEAM
🔍 Checking latest data for MDB...
📈 Fetching new data for MDB from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for MDB
🔍 Checking latest data for DKNG...
📈 Fetching new data for DKNG from 2025-01-30 to 2025-02-03...
✅ Updated 2 new entries for DKNG
🎉 Stock data update complete!


### Inserting The Past 5 years of data

In [3]:
import pandas as pd
from pymongo import MongoClient
df = pd.read_csv(r'C:\Users\Sezy\OneDrive\Personal_Projects\Trading_Algorithm\CSVFILES\top_10_stocks_2024.csv')

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")  # Default MongoDB URI
db = client["Stock_Data"]  # Replace with your desired database name
collection = db["Stock_5year"]  # Replace with your desired collection name

# Convert DataFrame to dictionary and insert into MongoDB
records = df.to_dict(orient='records')
collection.insert_many(records)

print(f"Inserted {len(records)} records into the database!")

Inserted 149292 records into the database!
